In [1]:
import numpy as np
import pandas as pd

In [3]:
D = pd.read_csv("DS_Sample_3.csv")

In [4]:
D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       188 non-null    object 
 1   AirDate     188 non-null    object 
 2   Rating      188 non-null    float64
 3   Num_Votes   188 non-null    int64  
 4   DirectedBy  188 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 7.5+ KB


In [5]:
D.describe()

,Rating,Num_Votes
count,188.000000,188.000000
mean,8.248404,2489.787234
std,0.578720,924.986433
min,6.600000,1630.000000
25%,7.800000,1918.000000
50%,8.200000,2295.000000
75%,8.600000,2769.000000
max,9.800000,9269.000000


In [6]:
D.head()

,Title,AirDate,Rating,Num_Votes,DirectedBy
0,Pilot,2005-03-24,7.5,4349,Ken Kwapis
1,Diversity Day,2005-03-29,8.3,4213,Ken Kwapis
2,Health Care,2005-04-05,7.8,3536,Ken Whittingham
3,The Alliance,2005-04-12,8.1,3428,Bryan Gordon
4,Basketball,2005-04-19,8.4,3745,Greg Daniels


In [9]:
year = D.AirDate.str[0:4]

In [10]:
year.max(), year.min()

('2013', '2005')

In [11]:
D['group'] = np.where(year <= '2007', 'A', np.where(year <= '2010', 'B',  'C') )

In [14]:
D[['group', 'AirDate']].groupby(by='group').AirDate.agg(['min', 'max'])

,min,max
group,,
A,2005-03-24,2007-11-15
B,2008-04-10,2010-12-09
C,2011-01-20,2013-05-16


In [15]:
movie3 = D.copy()

In [16]:
movie3

,Title,AirDate,Rating,Num_Votes,DirectedBy,group
0,Pilot,2005-03-24,7.5,4349,Ken Kwapis,A
1,Diversity Day,2005-03-29,8.3,4213,Ken Kwapis,A
2,Health Care,2005-04-05,7.8,3536,Ken Whittingham,A
3,The Alliance,2005-04-12,8.1,3428,Bryan Gordon,A
4,Basketball,2005-04-19,8.4,3745,Greg Daniels,A
...,...,...,...,...,...,...
183,Stairmageddon,2013-04-11,8.0,1743,Matt Sohn,C
184,Paper Airplane,2013-04-25,8.0,1748,Jesse Peretz,C
185,Livin' the Dream,2013-05-02,9.0,2439,Jeffrey Blitz,C
186,A.A.R.M.,2013-05-09,9.5,3401,David Rogers,C


In [18]:
movie3['success'] = movie3.Rating * movie3.Num_Votes

In [25]:
movie3.groupby("DirectedBy")[['success']].mean().sort_values("success", ascending=False).head(3)

,success
DirectedBy,
Ken Kwapis,32720.70
Bryan Gordon,31096.05
Paul Feig,28574.30


In [26]:
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols


In [29]:
movie3[['group', 'Num_Votes']]

,group,Num_Votes
0,A,4349
1,A,4213
2,A,3536
3,A,3428
4,A,3745
...,...,...
183,C,1743
184,C,1748
185,C,2439
186,C,3401


In [36]:
reg = ols('Num_Votes ~ C(group)', movie3[['group', 'Num_Votes']]).fit()

In [37]:
anova_lm(reg)

,df,sum_sq,mean_sq,F,PR(>F)
C(group),2.0,2.362951e+07,1.181476e+07,16.028213,3.805527e-07
Residual,185.0,1.363677e+08,7.371225e+05,NaN,NaN


In [38]:
from sklearn.preprocessing import OneHotEncoder

In [39]:
enc = OneHotEncoder()

In [42]:
onehot = enc.fit_transform(movie3[['group']]).toarray()

In [56]:
dummies = pd.DataFrame(onehot, columns = enc.categories_[0])
dummies

,A,B,C
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
183,0.0,0.0,1.0
184,0.0,0.0,1.0
185,0.0,0.0,1.0
186,0.0,0.0,1.0


In [60]:
#pd.concat([[, dummies]])
X = pd.concat( [movie3[['Num_Votes']], dummies ], axis = 1 )
X

,Num_Votes,A,B,C
0,4349,1.0,0.0,0.0
1,4213,1.0,0.0,0.0
2,3536,1.0,0.0,0.0
3,3428,1.0,0.0,0.0
4,3745,1.0,0.0,0.0
...,...,...,...,...
183,1743,0.0,0.0,1.0
184,1748,0.0,0.0,1.0
185,2439,0.0,0.0,1.0
186,3401,0.0,0.0,1.0


In [59]:
from sklearn.linear_model import LinearRegression

In [65]:
reg = LinearRegression ()

In [70]:
reg.fit(X[['Num_Votes','A','C']], movie3[['Rating']])

LinearRegression()

In [71]:
reg.predict( [
    [5000,0,1]
])

array([[9.03049081]])

In [72]:
reg2 = LinearRegression ()

In [73]:
reg.fit(X[['Num_Votes','A','B','C']], movie3[['Rating']])

LinearRegression()

In [74]:
reg.predict( [
    [5000,0,0, 1]
])

array([[9.03049081]])